In [3]:
import datetime, time
#from architecture import *
#model=load_model(N_NEIGHBORS)
#loaders=hoods(INPUT_DIR,N_NEIGHBORS)

import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

from egnn_pytorch import EGNN_Network
import torch
import torch.nn as nn

import math

#EGNN

class EGNNBlock(nn.Module):
    """todo: try to take head out here
    egnn_net --> layer norm --> ffn head"""
    def __init__(self, dim, depth,hidden_dim,dropout,
                 num_positions, num_tokens,
                 num_nearest_neighbors,
                 norm_coors):
        super().__init__()
        self.egnn = EGNN_Network(
            dim=dim, depth=depth, dropout=dropout,
            num_positions=num_positions,
            num_tokens=num_tokens,
            num_nearest_neighbors=num_nearest_neighbors,
            norm_coors=norm_coors
        )
        self.norm1 = nn.LayerNorm(dim)
        self.ffn = nn.Sequential(
            nn.Linear(dim, hidden_dim*dim),
            nn.PReLU(),# LU(),
            nn.Linear(hidden_dim*dim, dim),
        )
        self.norm2 = nn.LayerNorm(dim)
    def forward(self, z, x):
        (h_list, coords) = self.egnn(z, x)
        h = h_list # [B,N,dim]
        h2 = h
        h  = self.norm1(h + h2)
        h2 = self.ffn(h)
        h  = self.norm2(h + h2)
        return [h], coords

class StackedEGNN(nn.Module):
    """TODO understand depth"""
    def __init__(self, dim, depth, hidden_dim, dropout,
                 num_positions, num_tokens,
                 num_nearest_neighbors,
                 norm_coors):
        super().__init__()
        self.blocks = nn.ModuleList([
            EGNNBlock(dim, depth, hidden_dim, dropout,
                      num_positions, num_tokens,
                      num_nearest_neighbors,
                      norm_coors)
            for _ in range(1)
        ])
    def forward(self, z, x):
        coords = x
        h_list = None
        for block in self.blocks:
            if h_list is None:
                h_list, coords = block(z, x)
            else:
                h_list, coords = block(z, coords)
        return h_list, coords

# --- RBF with learnable cutoff ---
class LearnableRBF(nn.Module):
    """TODO change cutout"""
    def __init__(self, num_basis=16, cutoff=10.0):
        super().__init__()
        #self.pairwise = torch.norm(x.unsqueeze(1) - x.unsqueeze(0), dim=-1)
        self.cutoff = nn.Parameter(torch.tensor(cutoff))
        self.mu     = nn.Parameter(torch.linspace(0.0, 1.0, num_basis))
        self.gamma  = nn.Parameter(torch.tensor(12.0))
    
    def pairwise_distances(self, dist):
        return torch.norm(dist.unsqueeze(1) - dist.unsqueeze(0), dim=-1)

    
    def forward(self, dist):
        # dist: [B,N,N]
        dist = self.pairwise_distances(dist)
        mu = self.mu * self.cutoff     # [K]
        d  = dist.unsqueeze(-1)        # [B,N,N,1]
        return torch.exp(-self.gamma * (d - mu)**2)


#Attn. note encoding dropout of 0.03
#TODO: SPECIFy max len and dropout.
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=1000, dropout=0.03):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        pe = torch.zeros(max_len, embed_dim)
        pos = torch.arange(max_len, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, embed_dim, 2, dtype=torch.float)
                        * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(pos * div)
        cosp = torch.cos(pos * div)
        pe[:, 1::2] = cosp[:, : pe[:, 1::2].shape[1]]
        self.register_buffer('pe', pe.unsqueeze(1))
    def forward(self, x):
        return self.dropout(x + self.pe[: x.size(0)])

# --- Transformer‐style AttentionBlock ---
class AttentionBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, hidden_dim):
        super().__init__()
        self.encoding = PositionalEncoding(embed_dim)
        self.attn  = nn.MultiheadAttention(embed_dim, num_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.ffn   = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * hidden_dim),
            nn.PReLU(),
            nn.Linear(embed_dim * hidden_dim, embed_dim),
        )
        self.norm2 = nn.LayerNorm(embed_dim)
        
    def forward(self, x, key_padding_mask=None):
        # x: [seq_len, batch, embed_dim]
        x = self.encoding(x)
        a, _ = self.attn(x, x, x, key_padding_mask=key_padding_mask)
        x    = self.norm1(x + a)
        f    = self.ffn(x)
        x    = self.norm2(x + f)
        return x,_






import datetime, time
#from architecture import *
import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
from egnn_pytorch import EGNN
#from pkegnn import *
# 0) start timer
t0 = time.time()
N_NEIGHBORS = 100
BATCH_SIZE  =  1         # not safe to increase
PIN_MEMORY  = torch.cuda.is_available()
# reproducibility + device
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
conv=nn.Conv1d(1, 1, 7, padding=3).to(device)
# decide AMP only on GP0
use_amp = (device.type == "cuda")
if use_amp:
    scaler = GradScaler()
else:
    class DummyCM:
        def __enter__(self): pass
        def __exit__(self, *args): pass
    autocast = DummyCM
    scaler   = None

# 3) instantiate everything
dim, basis = 12, 8 #scale to 3,16 at least # dim must be divisible by 2
depth=2 #scale to 2, at least
hidden_dim=4
dropout=0.04
cutoff=20.0
num_neighbors=11



runid=datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "-noH"


from sklearn.neighbors import NearestNeighbors
from torch.utils.data import Dataset, DataLoader
import numpy as np, torch, glob

class InMemoryHoodDataset(Dataset):
    """
    Loads *.npz files, constructs fixed-size neighbourhoods around each
    site (anchor) and stores the result entirely in RAM.

    For a protein with S sites the shapes are
        z   : (S, N_NEIGHBORS)      int32
        pos : (S, N_NEIGHBORS, 3)   float32
        y   : (S,)                  float32
    """
    def __init__(self, paths, n_neighbors=N_NEIGHBORS, pin_memory=PIN_MEMORY):
        super().__init__()
        self.data = []
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm="brute")

        for p in paths:
            try:
                dat = np.load(p, allow_pickle=True)
                z_all   = dat["z"].astype(np.int32)        # (N,)
                pos_all = dat["pos"].astype(np.float32)    # (N,3)
                sites   = dat["sites"].astype(np.float32)  # (S,3)
                y       = dat["pks"].astype(np.float32)    # (S,)

                if len(sites) == 0:
                    continue  # skip empty entries

                nbrs.fit(pos_all)
                idx = nbrs.kneighbors(sites, return_distance=False)   # (S, N_NEIGHBORS)

                z_hood   = torch.from_numpy(z_all[idx])            # (S,N_NEIGHBORS)
                pos_hood = torch.from_numpy(pos_all[idx])          # (S,N_NEIGHBORS,3)
                y        = torch.from_numpy(y)                     # (S,)

                if pin_memory:
                    z_hood   = z_hood.pin_memory()
                    pos_hood = pos_hood.pin_memory()
                    y        = y.pin_memory()

                self.data.append((z_hood, pos_hood, y))
            except Exception as e:
                print(f"skipping {p}: {e}")

    def __len__(self):             return len(self.data)
    def __getitem__(self, idx):    return self.data[idx]

# ---------------------------------------------------------------------
# 2) collate function  -------------------------------------------------
# ---------------------------------------------------------------------
def pad_collate(batch):
    B      = len(batch)
    S_max  = max(item[0].shape[0] for item in batch)   # longest protein

    # --- allocate on CPU (no device=...) ---------------
    zs   = torch.zeros (B, S_max, N_NEIGHBORS ,   dtype=torch.int32 )
    pos  = torch.zeros (B, S_max, N_NEIGHBORS ,3, dtype=torch.float32)
    ys   = torch.full  ((B, S_max), float("nan"),  dtype=torch.float32)
    mask = torch.zeros (B, S_max,               dtype=torch.bool)

    for b,(z,pos_b,y) in enumerate(batch):
        S = z.shape[0]
        zs [b, :S] = z
        pos[b, :S] = pos_b
        ys [b, :S] = y
        mask[b, :S] = True

    return zs, pos, ys, mask


def count_params(model: torch.nn.Module, trainable_only: bool = True) -> int:
    """
    Returns the total number of (optionally trainable) parameters.
    """
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())


# ---------------------------------------------------------------------
# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------

# 0) parameters you might want to expose at the top of the script
# ---------------------------------------------------------------------

# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------
np.random.seed(0)
INPUTS_DIR="../../../data/pkegnn_INS/inputs/*.npz"
print("inputs dir:", INPUTS_DIR)
all_paths = sorted(glob.glob(INPUTS_DIR))
#print(all_paths)
#split=0.75
#t=int(len(all_paths) *.75) - 1

#v=int(len(all_paths) *.25)
np.random.shuffle(all_paths)
train_paths, val_paths = all_paths[:2] + all_paths[5:10], all_paths[20:50]
#print(train_paths[0],val_paths[0])
nconv=nn.Conv1d(N_NEIGHBORS,1,dim+basis).to(device)
train_ds = InMemoryHoodDataset(train_paths)
val_ds   = InMemoryHoodDataset(val_paths)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True,  collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)
val_loader   = DataLoader(val_ds  , batch_size=BATCH_SIZE,
                          shuffle=False, collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)


# ---------------------------------------------------------------------
# 4) model pieces ------------------------------------------------------
# ---------------------------------------------------------------------
egnn_net = StackedEGNN(dim=dim, depth=depth, hidden_dim=hidden_dim,
                       dropout=dropout, num_positions=1000, num_tokens=118,
                       num_nearest_neighbors=num_neighbors,
                       norm_coors=True).to(device)

rbf_layer = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device)
mha_layer = AttentionBlock(embed_dim=dim + basis,
                           num_heads=dim + basis,
                           hidden_dim=hidden_dim).to(device)
pred_head = nn.Linear(1, 1).to(device)
#pred_head2 = nn.Linear(1, 1).to(device)

protein_egnn=EGNN(dim=1,update_coors=True, norm_coors=True, norm_feats=True, fourier_features=6, valid_radius=8)
criterion = nn.L1Loss()
optimizer = torch.optim.AdamW(
    list(egnn_net.parameters()) +
    list(rbf_layer.parameters()) +
    list(mha_layer.parameters()) +
    list(conv.parameters()) +
    list(nconv.parameters()) +

    list(pred_head.parameters()) +
    list(protein_egnn.parameters()),
    lr=5e-3
)
modell=[egnn_net, protein_egnn, nconv,rbf_layer,mha_layer,pred_head,conv] ####
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=.99, patience=0, cooldown=0, min_lr=1e-8, verbose=False)
epochs = 30 # or whatever you like
total_all    = count_params(torch.nn.ModuleList(
                    modell))


print(f"Whole model          : {total_all:,}")

config={"runid": runid,
        "num_nbrs": N_NEIGHBORS,
        "num_nbrs_egnn": num_neighbors,
        "learning_rate": [op["lr"] for op in optimizer.param_groups], #net mha model rbf
        "dataset": (INPUTS_DIR,len(train_ds),len(val_ds)),
        "epochs": epochs,
        "dim": dim,
        "depth": depth,
        "basis": basis,
        "hidden_dim":hidden_dim,
        "num_heads":dim+basis, #!
        "dropout": [dropout, 0.03], #egnn p.enc. 
        "rbf cutoff": cutoff,
        "loss": criterion,
        "params":total_all}
        #"architect":str(modell).encode()}

print(config)
# ---------------------------------------------------------------------
# 5) forward for a *compressed* batch (R residues, N neighbours)
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# 5) single-path forward – no shape guessing, no branching
# ---------------------------------------------------------------------
def forward_residues(z_r, x_r):
    """
    z_r : (R, N)       int32   – atomic numbers for R residues
    x_r : (R, N, 3)    float32 – coordinates
    returns (R, dim + basis)   – per-residue embeddings
    """
    # ---------- EGNN ----------
    h_out, coords = egnn_net(z_r, x_r)          # h_out is [tensor] or tensor
    h = h_out[0] if isinstance(h_out, (list, tuple)) else h_out   # (R, N, dim)

    # ---------- RBF on *input* coords (already (R,N,3)) ----------
    #d   = torch.cdist(x_r, x_r)            # (R, N, N)
    rbf = rbf_layer(coords)                     # (R, N, N, basis)
    

    # ---------- concat & attention ----------
    h0  = h.transpose(1, 2)                # (R, dim,   N)
    r0  = rbf[:, 0].transpose(1, 2)        # (R, basis, N)
    tok = torch.cat((r0, h0), dim=1)       # (R, dim+basis, N)

    tok, _ = mha_layer(tok.permute(2, 0, 1))   # (N, R, C) → attn(+PE) #TODO suppress _
    tok=nconv(tok.to(device).permute(1,0,2)).to(device)
    #tok    = tok.permute(1, 0, 2).max(dim=1).values   # (R, C) max over neighbours
    return tok,     coords.mean(dim=1).unsqueeze(0)                               # (R, dim + basis)
                                         # (R, dim+basis)

# ---------------------------------------------------------------------
# 6) training / validation loop ---------------------------------------
# ---------------------------------------------------------------------
tloss, vloss = [], []
for epoch in range(epochs):
    # ======== TRAIN ========
    egnn_net.train(); rbf_layer.train(); mha_layer.train(); protein_egnn.train(); conv.train(); nconv.train();pred_head.train(); #p#red_head2.train(); 
    tr_losses = []

    for z, x, y, mask in train_loader:                 # z:(B,S,N)  mask:(B,S)
        # compress away padding →  (R, N), (R, N, 3), (R,)
        valid      = mask.view(-1)
        z_res      = z.view(-1, z.size(2))[valid].to(device)
        x_res      = x.view(-1, x.size(2), 3)[valid].to(device)
        y_res      = y.view(-1)[valid].to(device)

        optimizer.zero_grad()

        #model
        feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        #print(feats)
        preds = pred_head(feats.to(device))    
        t         = preds.to(device)[:,0].T.unsqueeze(2)         # (1, R, 1) on GPU
        centroids = centroids.to(device)  # (1, R, 3) on GPU

        protein_egnn = protein_egnn.to(device)             # make sure weights are there
        preds = protein_egnn(t, centroids)[0].permute(1,2,0)
        #preds = pred_head2(preds.to(device))
        #loss  = criterion(preds.flatten(), y_res)
        with autocast():
                preds=conv(preds).to(device)

                loss  = criterion(preds.flatten(), y_res)
            

        if use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()

        tr_losses.append(loss.item())
    tmean=np.mean(tr_losses)

    print(f"Epoch {epoch:3d} | train L1 = {tmean:.4f}")
    tloss.append(tmean.item())
    #elapsed_min = (time.time() - t0) / 60
    
    # ======== VALID ========
    egnn_net.eval(); rbf_layer.eval(); mha_layer.eval(); protein_egnn.eval(); conv.eval(); nconv.eval();pred_head.eval(); #pred_head2.eval(); 
    vl_losses = []

    with torch.no_grad():
        for z, x, y, mask in val_loader:
            valid   = mask.view(-1)
            z_res   = z.view(-1, z.size(2))[valid].to(device)
            x_res   = x.view(-1, x.size(2), 3)[valid].to(device)
            y_res   = y.view(-1)[valid].to(device)

                #model
            feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        
            preds = pred_head(feats.to(device))    
            t         = preds.to(device)[:,0].T.unsqueeze(2)         # (1, R, 1) on GPU
            centroids = centroids.to(device)  # (1, R, 3) on GPU

            protein_egnn = protein_egnn.to(device)             # make sure weights are there
            preds = protein_egnn(t, centroids)[0].permute(1,2,0)
            #preds = pred_head2(preds.to(device))
            with autocast():
                preds=conv(preds).to(device)

                loss  = criterion(preds.flatten(), y_res)
            vl_losses.append(loss) ###his is so inefficient. batch the proteins and for free bye cpu

    #elapsed_min = (time.time() - t0) / 60
    if vl_losses: #cpu
        vl=torch.mean(torch.stack(vl_losses))
        print(f"              |  val L1 = {vl.item():.4f}")
    else:
        print("E70")
        
    #L=torch.mean(torch.stack(vl_losses))
    scheduler.step(vl)
    vloss.append(vl.item())
    
    if epoch in [100,200,300,400,500,600,700,800,900]: #TODO doesnt work
        # 5) save a single timestamped checkpoint
        elapsed_min = (time.time() - t0) / 60
        timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        checkpoint = {
            "epoch":         epoch+1,
            "elapsed_min":   elapsed_min,
            "net":           egnn_net.state_dict(),
            "mha":           mha_layer.state_dict(),
            "model":         protein_egnn.state_dict(),
            "convs":         [conv.state_dict(), nconv.state_dict()],
            "lin":           pred_head.state_dict(),
            "rbf":           rbf_layer.state_dict(),
            "optimizer":     optimizer.state_dict(),
            "scheduler":     scheduler.state_dict(),
            "train_history": tloss,
            "val_history":   vloss,
            "config":        config,
        }
        torch.save(checkpoint, f"./checkpoints/benchmark_1049inputs_{runid}-checkpoint_{timestamp}.pt")
        #torch.save(checkpoint, f"./test-checkpoint_{timestamp}.pt")
        print(f"Saved checkpoint ./checkpoints/benchmark_1049inputs_{runid}-checkpoint_{timestamp}.pt", f"({elapsed_min:.1f} min)")
        #os.system("wandb sync --include-offline --sync-all wandb")
# 5) save a single timestamped checkpoint
elapsed_min = (time.time() - t0) / 60
timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint = {
    "epoch":         epoch+1,
    "elapsed_min":   elapsed_min,
    "net":           egnn_net.state_dict(),
    "mha":           mha_layer.state_dict(),
    "model":         protein_egnn.state_dict(),
    "convs":         [conv.state_dict(), nconv.state_dict()],
    "lin":           pred_head.state_dict(),
    "rbf":           rbf_layer.state_dict(),
    "optimizer":     optimizer.state_dict(),
    "scheduler":     scheduler.state_dict(),
    "train_history": tloss,
    "val_history":   vloss,
    "config":        config,
}
torch.save(checkpoint, f"../../../data/all_checkpoints/think_{runid}-checkpoint_{timestamp}.pt")
#torch.save(checkpoint, f"./test-checkpoint_{timestamp}.pt")
print(f"Saved checkpoint ../../../data/all_checkpoints/think_{runid}-checkpoint_{timestamp}.pt", elapsed_min)
#os.system("wandb sync --include-offline --sync-all wandb")


#from matplotlib import pyplot as plt
#plt.plot(vloss)
#plt.plot(tloss)


inputs dir: ../../../data/pkegnn_INS/inputs/*.npz
Whole model          : 32,510
{'runid': '20250722_064106-noH', 'num_nbrs': 100, 'num_nbrs_egnn': 11, 'learning_rate': [0.005], 'dataset': ('../../../data/pkegnn_INS/inputs/*.npz', 7, 30), 'epochs': 30, 'dim': 12, 'depth': 2, 'basis': 8, 'hidden_dim': 4, 'num_heads': 20, 'dropout': [0.04, 0.03], 'rbf cutoff': 20.0, 'loss': L1Loss(), 'params': 32510}
Epoch   0 | train L1 = 1.1970
              |  val L1 = 1.2643
Epoch   1 | train L1 = 1.1889


KeyboardInterrupt: 

In [4]:
import datetime, time
#from architecture import *
#model=load_model(N_NEIGHBORS)
#loaders=hoods(INPUT_DIR,N_NEIGHBORS)

import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

from egnn_pytorch import EGNN_Network
import torch
import torch.nn as nn

import math

#EGNN

class EGNNBlock(nn.Module):
    """todo: try to take head out here
    egnn_net --> layer norm --> ffn head"""
    def __init__(self, dim, depth,hidden_dim,dropout,
                 num_positions, num_tokens,
                 num_nearest_neighbors,
                 norm_coors):
        super().__init__()
        self.egnn = EGNN_Network(
            dim=dim, depth=depth, dropout=dropout,
            num_positions=num_positions,
            num_tokens=num_tokens,
            num_nearest_neighbors=num_nearest_neighbors,
            norm_coors=norm_coors
        )
        self.norm1 = nn.LayerNorm(dim)
        self.ffn = nn.Sequential(
            nn.Linear(dim, hidden_dim*dim),
            nn.PReLU(),# LU(),
            nn.Linear(hidden_dim*dim, dim),
        )
        self.norm2 = nn.LayerNorm(dim)
    def forward(self, z, x):
        (h_list, coords) = self.egnn(z, x)
        h = h_list # [B,N,dim]
        h2 = h
        h  = self.norm1(h + h2)
        h2 = self.ffn(h)
        h  = self.norm2(h + h2)
        return [h], coords

class StackedEGNN(nn.Module):
    """TODO understand depth"""
    def __init__(self, dim, depth, hidden_dim, dropout,
                 num_positions, num_tokens,
                 num_nearest_neighbors,
                 norm_coors):
        super().__init__()
        self.blocks = nn.ModuleList([
            EGNNBlock(dim, depth, hidden_dim, dropout,
                      num_positions, num_tokens,
                      num_nearest_neighbors,
                      norm_coors)
            for _ in range(1)
        ])
    def forward(self, z, x):
        coords = x
        h_list = None
        for block in self.blocks:
            if h_list is None:
                h_list, coords = block(z, x)
            else:
                h_list, coords = block(z, coords)
        return h_list, coords

# --- RBF with learnable cutoff ---
class LearnableRBF(nn.Module):
    """TODO change cutout"""
    def __init__(self, num_basis=16, cutoff=10.0):
        super().__init__()
        #self.pairwise = torch.norm(x.unsqueeze(1) - x.unsqueeze(0), dim=-1)
        self.cutoff = nn.Parameter(torch.tensor(cutoff))
        self.mu     = nn.Parameter(torch.linspace(0.0, 1.0, num_basis))
        self.gamma  = nn.Parameter(torch.tensor(12.0))
    
    def pairwise_distances(self, dist):
        return torch.norm(dist.unsqueeze(1) - dist.unsqueeze(0), dim=-1)

    
    def forward(self, dist):
        # dist: [B,N,N]
        dist = self.pairwise_distances(dist)
        mu = self.mu * self.cutoff     # [K]
        d  = dist.unsqueeze(-1)        # [B,N,N,1]
        return torch.exp(-self.gamma * (d - mu)**2)


#Attn. note encoding dropout of 0.03
#TODO: SPECIFy max len and dropout.
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=1000, dropout=0.03):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        pe = torch.zeros(max_len, embed_dim)
        pos = torch.arange(max_len, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, embed_dim, 2, dtype=torch.float)
                        * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(pos * div)
        cosp = torch.cos(pos * div)
        pe[:, 1::2] = cosp[:, : pe[:, 1::2].shape[1]]
        self.register_buffer('pe', pe.unsqueeze(1))
    def forward(self, x):
        return self.dropout(x + self.pe[: x.size(0)])

# --- Transformer‐style AttentionBlock ---
class AttentionBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, hidden_dim):
        super().__init__()
        self.encoding = PositionalEncoding(embed_dim)
        self.attn  = nn.MultiheadAttention(embed_dim, num_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.ffn   = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * hidden_dim),
            nn.PReLU(),
            nn.Linear(embed_dim * hidden_dim, embed_dim),
        )
        self.norm2 = nn.LayerNorm(embed_dim)
        
    def forward(self, x, key_padding_mask=None):
        # x: [seq_len, batch, embed_dim]
        x = self.encoding(x)
        a, _ = self.attn(x, x, x, key_padding_mask=key_padding_mask)
        x    = self.norm1(x + a)
        f    = self.ffn(x)
        x    = self.norm2(x + f)
        return x,_






import datetime, time
#from architecture import *
import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
from egnn_pytorch import EGNN
#from pkegnn import *
# 0) start timer
t0 = time.time()
N_NEIGHBORS = 100
BATCH_SIZE  =  1         # not safe to increase
PIN_MEMORY  = torch.cuda.is_available()
# reproducibility + device
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
conv=nn.Conv1d(1, 1, 7, padding=3).to(device)
# decide AMP only on GP0
use_amp = (device.type == "cuda")
if use_amp:
    scaler = GradScaler()
else:
    class DummyCM:
        def __enter__(self): pass
        def __exit__(self, *args): pass
    autocast = DummyCM
    scaler   = None

# 3) instantiate everything
dim, basis = 12, 8 #scale to 3,16 at least # dim must be divisible by 2
depth=2 #scale to 2, at least
hidden_dim=4
dropout=0.04
cutoff=20.0
num_neighbors=11



runid=datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "-noH"


from sklearn.neighbors import NearestNeighbors
from torch.utils.data import Dataset, DataLoader
import numpy as np, torch, glob

class InMemoryHoodDataset(Dataset):
    """
    Loads *.npz files, constructs fixed-size neighbourhoods around each
    site (anchor) and stores the result entirely in RAM.

    For a protein with S sites the shapes are
        z   : (S, N_NEIGHBORS)      int32
        pos : (S, N_NEIGHBORS, 3)   float32
        y   : (S,)                  float32
    """
    def __init__(self, paths, n_neighbors=N_NEIGHBORS, pin_memory=PIN_MEMORY):
        super().__init__()
        self.data = []
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm="brute")

        for p in paths:
            try:
                dat = np.load(p, allow_pickle=True)
                z_all   = dat["z"].astype(np.int32)        # (N,)
                pos_all = dat["pos"].astype(np.float32)    # (N,3)
                sites   = dat["sites"].astype(np.float32)  # (S,3)
                y       = dat["pks"].astype(np.float32)    # (S,)

                if len(sites) == 0:
                    continue  # skip empty entries

                nbrs.fit(pos_all)
                idx = nbrs.kneighbors(sites, return_distance=False)   # (S, N_NEIGHBORS)

                z_hood   = torch.from_numpy(z_all[idx])            # (S,N_NEIGHBORS)
                pos_hood = torch.from_numpy(pos_all[idx])          # (S,N_NEIGHBORS,3)
                y        = torch.from_numpy(y)                     # (S,)

                if pin_memory:
                    z_hood   = z_hood.pin_memory()
                    pos_hood = pos_hood.pin_memory()
                    y        = y.pin_memory()

                self.data.append((z_hood, pos_hood, y))
            except Exception as e:
                print(f"skipping {p}: {e}")

    def __len__(self):             return len(self.data)
    def __getitem__(self, idx):    return self.data[idx]

# ---------------------------------------------------------------------
# 2) collate function  -------------------------------------------------
# ---------------------------------------------------------------------
def pad_collate(batch):
    B      = len(batch)
    S_max  = max(item[0].shape[0] for item in batch)   # longest protein

    # --- allocate on CPU (no device=...) ---------------
    zs   = torch.zeros (B, S_max, N_NEIGHBORS ,   dtype=torch.int32 )
    pos  = torch.zeros (B, S_max, N_NEIGHBORS ,3, dtype=torch.float32)
    ys   = torch.full  ((B, S_max), float("nan"),  dtype=torch.float32)
    mask = torch.zeros (B, S_max,               dtype=torch.bool)

    for b,(z,pos_b,y) in enumerate(batch):
        S = z.shape[0]
        zs [b, :S] = z
        pos[b, :S] = pos_b
        ys [b, :S] = y
        mask[b, :S] = True

    return zs, pos, ys, mask


def count_params(model: torch.nn.Module, trainable_only: bool = True) -> int:
    """
    Returns the total number of (optionally trainable) parameters.
    """
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())


# ---------------------------------------------------------------------
# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------

# 0) parameters you might want to expose at the top of the script
# ---------------------------------------------------------------------

# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------
np.random.seed(0)
INPUTS_DIR="../../../data/pkegnn_INS/inputs/*.npz"
print("inputs dir:", INPUTS_DIR)
all_paths = sorted(glob.glob(INPUTS_DIR))
#print(all_paths)
#split=0.75
#t=int(len(all_paths) *.75) - 1

#v=int(len(all_paths) *.25)
np.random.shuffle(all_paths)
train_paths, val_paths = all_paths[:2] + all_paths[5:10], all_paths[20:50]
#print(train_paths[0],val_paths[0])
nconv=nn.Conv1d(N_NEIGHBORS,1,dim+basis).to(device)
train_ds = InMemoryHoodDataset(train_paths)
val_ds   = InMemoryHoodDataset(val_paths)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True,  collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)
val_loader   = DataLoader(val_ds  , batch_size=BATCH_SIZE,
                          shuffle=False, collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)


# ---------------------------------------------------------------------
# 4) model pieces ------------------------------------------------------
# ---------------------------------------------------------------------
egnn_net = StackedEGNN(dim=dim, depth=depth, hidden_dim=hidden_dim,
                       dropout=dropout, num_positions=1000, num_tokens=118,
                       num_nearest_neighbors=num_neighbors,
                       norm_coors=True).to(device)

rbf_layer = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device)
mha_layer = AttentionBlock(embed_dim=dim + basis,
                           num_heads=dim + basis,
                           hidden_dim=hidden_dim).to(device)
pred_head = nn.Linear(1, 1).to(device)
#pred_head2 = nn.Linear(1, 1).to(device)

protein_egnn=EGNN(dim=1,update_coors=True, norm_coors=True, norm_feats=True, fourier_features=6, valid_radius=8)
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(
    list(egnn_net.parameters()) +
    list(rbf_layer.parameters()) +
    list(mha_layer.parameters()) +
    list(conv.parameters()) +
    list(nconv.parameters()) +

    list(pred_head.parameters()) +
    list(protein_egnn.parameters()),
    lr=5e-3
)
modell=[egnn_net, protein_egnn, nconv,rbf_layer,mha_layer,pred_head,conv] ####
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=.99, patience=0, cooldown=0, min_lr=1e-8, verbose=False)
epochs = 30 # or whatever you like
total_all    = count_params(torch.nn.ModuleList(
                    modell))


print(f"Whole model          : {total_all:,}")

config={"runid": runid,
        "num_nbrs": N_NEIGHBORS,
        "num_nbrs_egnn": num_neighbors,
        "learning_rate": [op["lr"] for op in optimizer.param_groups], #net mha model rbf
        "dataset": (INPUTS_DIR,len(train_ds),len(val_ds)),
        "epochs": epochs,
        "dim": dim,
        "depth": depth,
        "basis": basis,
        "hidden_dim":hidden_dim,
        "num_heads":dim+basis, #!
        "dropout": [dropout, 0.03], #egnn p.enc. 
        "rbf cutoff": cutoff,
        "loss": criterion,
        "params":total_all}
        #"architect":str(modell).encode()}

print(config)
# ---------------------------------------------------------------------
# 5) forward for a *compressed* batch (R residues, N neighbours)
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# 5) single-path forward – no shape guessing, no branching
# ---------------------------------------------------------------------
def forward_residues(z_r, x_r):
    """
    z_r : (R, N)       int32   – atomic numbers for R residues
    x_r : (R, N, 3)    float32 – coordinates
    returns (R, dim + basis)   – per-residue embeddings
    """
    # ---------- EGNN ----------
    h_out, coords = egnn_net(z_r, x_r)          # h_out is [tensor] or tensor
    h = h_out[0] if isinstance(h_out, (list, tuple)) else h_out   # (R, N, dim)

    # ---------- RBF on *input* coords (already (R,N,3)) ----------
    #d   = torch.cdist(x_r, x_r)            # (R, N, N)
    rbf = rbf_layer(coords)                     # (R, N, N, basis)
    

    # ---------- concat & attention ----------
    h0  = h.transpose(1, 2)                # (R, dim,   N)
    r0  = rbf[:, 0].transpose(1, 2)        # (R, basis, N)
    tok = torch.cat((r0, h0), dim=1)       # (R, dim+basis, N)

    tok, _ = mha_layer(tok.permute(2, 0, 1))   # (N, R, C) → attn(+PE) #TODO suppress _
    tok=nconv(tok.to(device).permute(1,0,2)).to(device)
    #tok    = tok.permute(1, 0, 2).max(dim=1).values   # (R, C) max over neighbours
    return tok,     coords.mean(dim=1).unsqueeze(0)                               # (R, dim + basis)
                                         # (R, dim+basis)

# ---------------------------------------------------------------------
# 6) training / validation loop ---------------------------------------
# ---------------------------------------------------------------------
tloss, vloss = [], []
for epoch in range(epochs):
    # ======== TRAIN ========
    egnn_net.train(); rbf_layer.train(); mha_layer.train(); protein_egnn.train(); conv.train(); nconv.train();pred_head.train(); #p#red_head2.train(); 
    tr_losses = []

    for z, x, y, mask in train_loader:                 # z:(B,S,N)  mask:(B,S)
        # compress away padding →  (R, N), (R, N, 3), (R,)
        valid      = mask.view(-1)
        z_res      = z.view(-1, z.size(2))[valid].to(device)
        x_res      = x.view(-1, x.size(2), 3)[valid].to(device)
        y_res      = y.view(-1)[valid].to(device)

        optimizer.zero_grad()

        #model
        feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        #print(feats)
        preds = pred_head(feats.to(device))    
        t         = preds.to(device)[:,0].T.unsqueeze(2)         # (1, R, 1) on GPU
        centroids = centroids.to(device)  # (1, R, 3) on GPU

        protein_egnn = protein_egnn.to(device)             # make sure weights are there
        preds = protein_egnn(t, centroids)[0].permute(1,2,0)
        #preds = pred_head2(preds.to(device))
        #loss  = criterion(preds.flatten(), y_res)
        with autocast():
                preds=conv(preds).to(device)

                loss  = criterion(preds.flatten(), y_res)
            

        if use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()

        tr_losses.append(loss.item())
    tmean=np.mean(tr_losses)

    print(f"Epoch {epoch:3d} | train L1 = {tmean:.4f}")
    tloss.append(tmean.item())
    #elapsed_min = (time.time() - t0) / 60
    
    # ======== VALID ========
    egnn_net.eval(); rbf_layer.eval(); mha_layer.eval(); protein_egnn.eval(); conv.eval(); nconv.eval();pred_head.eval(); #pred_head2.eval(); 
    vl_losses = []

    with torch.no_grad():
        for z, x, y, mask in val_loader:
            valid   = mask.view(-1)
            z_res   = z.view(-1, z.size(2))[valid].to(device)
            x_res   = x.view(-1, x.size(2), 3)[valid].to(device)
            y_res   = y.view(-1)[valid].to(device)

                #model
            feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        
            preds = pred_head(feats.to(device))    
            t         = preds.to(device)[:,0].T.unsqueeze(2)         # (1, R, 1) on GPU
            centroids = centroids.to(device)  # (1, R, 3) on GPU

            protein_egnn = protein_egnn.to(device)             # make sure weights are there
            preds = protein_egnn(t, centroids)[0].permute(1,2,0)
            #preds = pred_head2(preds.to(device))
            with autocast():
                preds=conv(preds).to(device)

                loss  = criterion(preds.flatten(), y_res)
            vl_losses.append(loss) ###his is so inefficient. batch the proteins and for free bye cpu

    #elapsed_min = (time.time() - t0) / 60
    if vl_losses: #cpu
        vl=torch.mean(torch.stack(vl_losses))
        print(f"              |  val L1 = {vl.item():.4f}")
    else:
        print("E70")
        
    #L=torch.mean(torch.stack(vl_losses))
    scheduler.step(vl)
    vloss.append(vl.item())
    
    if epoch in [100,200,300,400,500,600,700,800,900]: #TODO doesnt work
        # 5) save a single timestamped checkpoint
        elapsed_min = (time.time() - t0) / 60
        timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        checkpoint = {
            "epoch":         epoch+1,
            "elapsed_min":   elapsed_min,
            "net":           egnn_net.state_dict(),
            "mha":           mha_layer.state_dict(),
            "model":         protein_egnn.state_dict(),
            "convs":         [conv.state_dict(), nconv.state_dict()],
            "lin":           pred_head.state_dict(),
            "rbf":           rbf_layer.state_dict(),
            "optimizer":     optimizer.state_dict(),
            "scheduler":     scheduler.state_dict(),
            "train_history": tloss,
            "val_history":   vloss,
            "config":        config,
        }
        torch.save(checkpoint, f"./checkpoints/benchmark_1049inputs_{runid}-checkpoint_{timestamp}.pt")
        #torch.save(checkpoint, f"./test-checkpoint_{timestamp}.pt")
        print(f"Saved checkpoint ./checkpoints/benchmark_1049inputs_{runid}-checkpoint_{timestamp}.pt", f"({elapsed_min:.1f} min)")
        #os.system("wandb sync --include-offline --sync-all wandb")
# 5) save a single timestamped checkpoint
elapsed_min = (time.time() - t0) / 60
timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint = {
    "epoch":         epoch+1,
    "elapsed_min":   elapsed_min,
    "net":           egnn_net.state_dict(),
    "mha":           mha_layer.state_dict(),
    "model":         protein_egnn.state_dict(),
    "convs":         [conv.state_dict(), nconv.state_dict()],
    "lin":           pred_head.state_dict(),
    "rbf":           rbf_layer.state_dict(),
    "optimizer":     optimizer.state_dict(),
    "scheduler":     scheduler.state_dict(),
    "train_history": tloss,
    "val_history":   vloss,
    "config":        config,
}
torch.save(checkpoint, f"../../../data/all_checkpoints/think_{runid}-checkpoint_{timestamp}.pt")
#torch.save(checkpoint, f"./test-checkpoint_{timestamp}.pt")
print(f"Saved checkpoint ../../../data/all_checkpoints/think_{runid}-checkpoint_{timestamp}.pt", elapsed_min)
#os.system("wandb sync --include-offline --sync-all wandb")


#from matplotlib import pyplot as plt
#plt.plot(vloss)
#plt.plot(tloss)


inputs dir: ../../../data/pkegnn_INS/inputs/*.npz
Whole model          : 32,510
{'runid': '20250722_064258-noH', 'num_nbrs': 100, 'num_nbrs_egnn': 11, 'learning_rate': [0.005], 'dataset': ('../../../data/pkegnn_INS/inputs/*.npz', 7, 30), 'epochs': 30, 'dim': 12, 'depth': 2, 'basis': 8, 'hidden_dim': 4, 'num_heads': 20, 'dropout': [0.04, 0.03], 'rbf cutoff': 20.0, 'loss': MSELoss(), 'params': 32510}
Epoch   0 | train L1 = 3.4039
              |  val L1 = 3.5645
Epoch   1 | train L1 = 3.3587
              |  val L1 = 3.5654
Epoch   2 | train L1 = 3.4136
              |  val L1 = 3.5626
Epoch   3 | train L1 = 3.3351
              |  val L1 = 3.5361


KeyboardInterrupt: 

In [7]:
import datetime, time
#from architecture import *
#model=load_model(N_NEIGHBORS)
#loaders=hoods(INPUT_DIR,N_NEIGHBORS)

import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

from egnn_pytorch import EGNN_Network
import torch
import torch.nn as nn

import math

#EGNN

class EGNNBlock(nn.Module):
    """todo: try to take head out here
    egnn_net --> layer norm --> ffn head"""
    def __init__(self, dim, depth,hidden_dim,dropout,
                 num_positions, num_tokens,
                 num_nearest_neighbors,
                 norm_coors):
        super().__init__()
        self.egnn = EGNN_Network(
            dim=dim, depth=depth, dropout=dropout,
            num_positions=num_positions,
            num_tokens=num_tokens,
            num_nearest_neighbors=num_nearest_neighbors,
            norm_coors=norm_coors
        )
        self.norm1 = nn.LayerNorm(dim)
        self.ffn = nn.Sequential(
            nn.Linear(dim, hidden_dim*dim),
            nn.PReLU(),# LU(),
            nn.Linear(hidden_dim*dim, dim),
        )
        self.norm2 = nn.LayerNorm(dim)
    def forward(self, z, x):
        (h_list, coords) = self.egnn(z, x)
        h = h_list # [B,N,dim]
        h2 = h
        h  = self.norm1(h + h2)
        h2 = self.ffn(h)
        h  = self.norm2(h + h2)
        return [h], coords

class StackedEGNN(nn.Module):
    """TODO understand depth"""
    def __init__(self, dim, depth, hidden_dim, dropout,
                 num_positions, num_tokens,
                 num_nearest_neighbors,
                 norm_coors):
        super().__init__()
        self.blocks = nn.ModuleList([
            EGNNBlock(dim, depth, hidden_dim, dropout,
                      num_positions, num_tokens,
                      num_nearest_neighbors,
                      norm_coors)
            for _ in range(1)
        ])
    def forward(self, z, x):
        coords = x
        h_list = None
        for block in self.blocks:
            if h_list is None:
                h_list, coords = block(z, x)
            else:
                h_list, coords = block(z, coords)
        return h_list, coords

# --- RBF with learnable cutoff ---
class LearnableRBF(nn.Module):
    """TODO change cutout"""
    def __init__(self, num_basis=16, cutoff=10.0):
        super().__init__()
        #self.pairwise = torch.norm(x.unsqueeze(1) - x.unsqueeze(0), dim=-1)
        self.cutoff = nn.Parameter(torch.tensor(cutoff))
        self.mu     = nn.Parameter(torch.linspace(0.0, 1.0, num_basis))
        self.gamma  = nn.Parameter(torch.tensor(12.0))
    
    def pairwise_distances(self, dist):
        return torch.norm(dist.unsqueeze(1) - dist.unsqueeze(0), dim=-1)

    
    def forward(self, dist):
        # dist: [B,N,N]
        dist = self.pairwise_distances(dist)
        mu = self.mu * self.cutoff     # [K]
        d  = dist.unsqueeze(-1)        # [B,N,N,1]
        return torch.exp(-self.gamma * (d - mu)**2)


#Attn. note encoding dropout of 0.03
#TODO: SPECIFy max len and dropout.
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=1000, dropout=0.03):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        pe = torch.zeros(max_len, embed_dim)
        pos = torch.arange(max_len, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, embed_dim, 2, dtype=torch.float)
                        * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(pos * div)
        cosp = torch.cos(pos * div)
        pe[:, 1::2] = cosp[:, : pe[:, 1::2].shape[1]]
        self.register_buffer('pe', pe.unsqueeze(1))
    def forward(self, x):
        return self.dropout(x + self.pe[: x.size(0)])

# --- Transformer‐style AttentionBlock ---
class AttentionBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, hidden_dim):
        super().__init__()
        self.encoding = PositionalEncoding(embed_dim)
        self.attn  = nn.MultiheadAttention(embed_dim, num_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.ffn   = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * hidden_dim),
            nn.PReLU(),
            nn.Linear(embed_dim * hidden_dim, embed_dim),
        )
        self.norm2 = nn.LayerNorm(embed_dim)
        
    def forward(self, x, key_padding_mask=None):
        # x: [seq_len, batch, embed_dim]
        x = self.encoding(x)
        a, _ = self.attn(x, x, x, key_padding_mask=key_padding_mask)
        x    = self.norm1(x + a)
        f    = self.ffn(x)
        x    = self.norm2(x + f)
        return x,_






import datetime, time
#from architecture import *
import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
from egnn_pytorch import EGNN
#from pkegnn import *
# 0) start timer
t0 = time.time()
N_NEIGHBORS = 100
BATCH_SIZE  =  1         # not safe to increase
PIN_MEMORY  = torch.cuda.is_available()
# reproducibility + device
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
conv=nn.Conv1d(1, 1, 7, padding=3).to(device)
# decide AMP only on GP0
use_amp = (device.type == "cuda")
if use_amp:
    scaler = GradScaler()
else:
    class DummyCM:
        def __enter__(self): pass
        def __exit__(self, *args): pass
    autocast = DummyCM
    scaler   = None

# 3) instantiate everything
dim, basis = 2, 3 #scale to 3,16 at least # dim must be divisible by 2
depth=2 #scale to 2, at least
hidden_dim=4
dropout=0.01
cutoff=20.0
num_neighbors=11



runid=datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "-noH"


from sklearn.neighbors import NearestNeighbors
from torch.utils.data import Dataset, DataLoader
import numpy as np, torch, glob

class InMemoryHoodDataset(Dataset):
    """
    Loads *.npz files, constructs fixed-size neighbourhoods around each
    site (anchor) and stores the result entirely in RAM.

    For a protein with S sites the shapes are
        z   : (S, N_NEIGHBORS)      int32
        pos : (S, N_NEIGHBORS, 3)   float32
        y   : (S,)                  float32
    """
    def __init__(self, paths, n_neighbors=N_NEIGHBORS, pin_memory=PIN_MEMORY):
        super().__init__()
        self.data = []
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm="brute")

        for p in paths:
            try:
                dat = np.load(p, allow_pickle=True)
                z_all   = dat["z"].astype(np.int32)        # (N,)
                pos_all = dat["pos"].astype(np.float32)    # (N,3)
                sites   = dat["sites"].astype(np.float32)  # (S,3)
                y       = dat["pks"].astype(np.float32)    # (S,)

                if len(sites) == 0:
                    continue  # skip empty entries

                nbrs.fit(pos_all)
                idx = nbrs.kneighbors(sites, return_distance=False)   # (S, N_NEIGHBORS)

                z_hood   = torch.from_numpy(z_all[idx])            # (S,N_NEIGHBORS)
                pos_hood = torch.from_numpy(pos_all[idx])          # (S,N_NEIGHBORS,3)
                y        = torch.from_numpy(y)                     # (S,)

                if pin_memory:
                    z_hood   = z_hood.pin_memory()
                    pos_hood = pos_hood.pin_memory()
                    y        = y.pin_memory()

                self.data.append((z_hood, pos_hood, y))
            except Exception as e:
                print(f"skipping {p}: {e}")

    def __len__(self):             return len(self.data)
    def __getitem__(self, idx):    return self.data[idx]

# ---------------------------------------------------------------------
# 2) collate function  -------------------------------------------------
# ---------------------------------------------------------------------
def pad_collate(batch):
    B      = len(batch)
    S_max  = max(item[0].shape[0] for item in batch)   # longest protein

    # --- allocate on CPU (no device=...) ---------------
    zs   = torch.zeros (B, S_max, N_NEIGHBORS ,   dtype=torch.int32 )
    pos  = torch.zeros (B, S_max, N_NEIGHBORS ,3, dtype=torch.float32)
    ys   = torch.full  ((B, S_max), float("nan"),  dtype=torch.float32)
    mask = torch.zeros (B, S_max,               dtype=torch.bool)

    for b,(z,pos_b,y) in enumerate(batch):
        S = z.shape[0]
        zs [b, :S] = z
        pos[b, :S] = pos_b
        ys [b, :S] = y
        mask[b, :S] = True

    return zs, pos, ys, mask


def count_params(model: torch.nn.Module, trainable_only: bool = True) -> int:
    """
    Returns the total number of (optionally trainable) parameters.
    """
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())


# ---------------------------------------------------------------------
# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------

# 0) parameters you might want to expose at the top of the script
# ---------------------------------------------------------------------

# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------
np.random.seed(0)
INPUTS_DIR="../../../data/pkegnn_INS/inputs/*.npz"
print("inputs dir:", INPUTS_DIR)
all_paths = sorted(glob.glob(INPUTS_DIR))
#print(all_paths)
#split=0.75
#t=int(len(all_paths) *.75) - 1

#v=int(len(all_paths) *.25)
np.random.shuffle(all_paths)
train_paths, val_paths = all_paths[:2] + all_paths[5:10], all_paths[20:50]
#print(train_paths[0],val_paths[0])
nconv=nn.Conv1d(N_NEIGHBORS,1,dim+basis).to(device)
train_ds = InMemoryHoodDataset(train_paths)
val_ds   = InMemoryHoodDataset(val_paths)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True,  collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)
val_loader   = DataLoader(val_ds  , batch_size=BATCH_SIZE,
                          shuffle=False, collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)


# ---------------------------------------------------------------------
# 4) model pieces ------------------------------------------------------
# ---------------------------------------------------------------------
egnn_net = StackedEGNN(dim=dim, depth=depth, hidden_dim=hidden_dim,
                       dropout=dropout, num_positions=1000, num_tokens=118,
                       num_nearest_neighbors=num_neighbors,
                       norm_coors=True).to(device)

rbf_layer = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device)
mha_layer = AttentionBlock(embed_dim=dim + basis,
                           num_heads=dim + basis,
                           hidden_dim=hidden_dim).to(device)
pred_head = nn.Linear(1, 1).to(device)
#pred_head2 = nn.Linear(1, 1).to(device)

protein_egnn=EGNN(dim=1,update_coors=True, norm_coors=True, norm_feats=True, fourier_features=6, valid_radius=8)
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(
    list(egnn_net.parameters()) +
    list(rbf_layer.parameters()) +
    list(mha_layer.parameters()) +
    list(conv.parameters()) +
    list(nconv.parameters()) +

    list(pred_head.parameters()) +
    list(protein_egnn.parameters()),
    lr=5e-3
)
modell=[egnn_net, protein_egnn, nconv,rbf_layer,mha_layer,pred_head,conv] ####
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=.99, patience=0, cooldown=0, min_lr=1e-8, verbose=False)
epochs = 30 # or whatever you like
total_all    = count_params(torch.nn.ModuleList(
                    modell))


print(f"Whole model          : {total_all:,}")

config={"runid": runid,
        "num_nbrs": N_NEIGHBORS,
        "num_nbrs_egnn": num_neighbors,
        "learning_rate": [op["lr"] for op in optimizer.param_groups], #net mha model rbf
        "dataset": (INPUTS_DIR,len(train_ds),len(val_ds)),
        "epochs": epochs,
        "dim": dim,
        "depth": depth,
        "basis": basis,
        "hidden_dim":hidden_dim,
        "num_heads":dim+basis, #!
        "dropout": [dropout, 0.03], #egnn p.enc. 
        "rbf cutoff": cutoff,
        "loss": criterion,
        "params":total_all}
        #"architect":str(modell).encode()}

print(config)
# ---------------------------------------------------------------------
# 5) forward for a *compressed* batch (R residues, N neighbours)
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# 5) single-path forward – no shape guessing, no branching
# ---------------------------------------------------------------------
def forward_residues(z_r, x_r):
    """
    z_r : (R, N)       int32   – atomic numbers for R residues
    x_r : (R, N, 3)    float32 – coordinates
    returns (R, dim + basis)   – per-residue embeddings
    """
    # ---------- EGNN ----------
    h_out, coords = egnn_net(z_r, x_r)          # h_out is [tensor] or tensor
    h = h_out[0] if isinstance(h_out, (list, tuple)) else h_out   # (R, N, dim)

    # ---------- RBF on *input* coords (already (R,N,3)) ----------
    #d   = torch.cdist(x_r, x_r)            # (R, N, N)
    rbf = rbf_layer(coords)                     # (R, N, N, basis)
    

    # ---------- concat & attention ----------
    h0  = h.transpose(1, 2)                # (R, dim,   N)
    r0  = rbf[:, 0].transpose(1, 2)        # (R, basis, N)
    tok = torch.cat((r0, h0), dim=1)       # (R, dim+basis, N)

    tok, _ = mha_layer(tok.permute(2, 0, 1))   # (N, R, C) → attn(+PE) #TODO suppress _
    tok=nconv(tok.to(device).permute(1,0,2)).to(device)
    #tok    = tok.permute(1, 0, 2).max(dim=1).values   # (R, C) max over neighbours
    return tok,     coords.mean(dim=1).unsqueeze(0)                               # (R, dim + basis)
                                         # (R, dim+basis)

# ---------------------------------------------------------------------
# 6) training / validation loop ---------------------------------------
# ---------------------------------------------------------------------
tloss, vloss = [], []
for epoch in range(epochs):
    # ======== TRAIN ========
    egnn_net.train(); rbf_layer.train(); mha_layer.train(); protein_egnn.train(); conv.train(); nconv.train();pred_head.train(); #p#red_head2.train(); 
    tr_losses = []

    for z, x, y, mask in train_loader:                 # z:(B,S,N)  mask:(B,S)
        # compress away padding →  (R, N), (R, N, 3), (R,)
        valid      = mask.view(-1)
        z_res      = z.view(-1, z.size(2))[valid].to(device)
        x_res      = x.view(-1, x.size(2), 3)[valid].to(device)
        y_res      = y.view(-1)[valid].to(device)

        optimizer.zero_grad()

        #model
        feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        #print(feats)
        preds = pred_head(feats.to(device))    
        t         = preds.to(device)[:,0].T.unsqueeze(2)         # (1, R, 1) on GPU
        centroids = centroids.to(device)  # (1, R, 3) on GPU

        protein_egnn = protein_egnn.to(device)             # make sure weights are there
        preds = protein_egnn(t, centroids)[0].permute(1,2,0)
        #preds = pred_head2(preds.to(device))
        #loss  = criterion(preds.flatten(), y_res)
        with autocast():
                preds=conv(preds).to(device)

                loss  = criterion(preds.flatten(), y_res)
            

        if use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()

        tr_losses.append(loss.item())
    tmean=np.mean(tr_losses)

    print(f"Epoch {epoch:3d} | train L1 = {tmean:.4f}")
    tloss.append(tmean.item())
    #elapsed_min = (time.time() - t0) / 60
    
    # ======== VALID ========
    egnn_net.eval(); rbf_layer.eval(); mha_layer.eval(); protein_egnn.eval(); conv.eval(); nconv.eval();pred_head.eval(); #pred_head2.eval(); 
    vl_losses = []

    with torch.no_grad():
        for z, x, y, mask in val_loader:
            valid   = mask.view(-1)
            z_res   = z.view(-1, z.size(2))[valid].to(device)
            x_res   = x.view(-1, x.size(2), 3)[valid].to(device)
            y_res   = y.view(-1)[valid].to(device)

                #model
            feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        
            preds = pred_head(feats.to(device))    
            t         = preds.to(device)[:,0].T.unsqueeze(2)         # (1, R, 1) on GPU
            centroids = centroids.to(device)  # (1, R, 3) on GPU

            protein_egnn = protein_egnn.to(device)             # make sure weights are there
            preds = protein_egnn(t, centroids)[0].permute(1,2,0)
            #preds = pred_head2(preds.to(device))
            with autocast():
                preds=conv(preds).to(device)

                loss  = criterion(preds.flatten(), y_res)
            vl_losses.append(loss) ###his is so inefficient. batch the proteins and for free bye cpu

    #elapsed_min = (time.time() - t0) / 60
    if vl_losses: #cpu
        vl=torch.mean(torch.stack(vl_losses))
        print(f"              |  val L1 = {vl.item():.4f}")
    else:
        print("E70")
        
    #L=torch.mean(torch.stack(vl_losses))
    scheduler.step(vl)
    vloss.append(vl.item())
    
    if epoch in [100,200,300,400,500,600,700,800,900]: #TODO doesnt work
        # 5) save a single timestamped checkpoint
        elapsed_min = (time.time() - t0) / 60
        timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        checkpoint = {
            "epoch":         epoch+1,
            "elapsed_min":   elapsed_min,
            "net":           egnn_net.state_dict(),
            "mha":           mha_layer.state_dict(),
            "model":         protein_egnn.state_dict(),
            "convs":         [conv.state_dict(), nconv.state_dict()],
            "lin":           pred_head.state_dict(),
            "rbf":           rbf_layer.state_dict(),
            "optimizer":     optimizer.state_dict(),
            "scheduler":     scheduler.state_dict(),
            "train_history": tloss,
            "val_history":   vloss,
            "config":        config,
        }
        torch.save(checkpoint, f"./checkpoints/benchmark_1049inputs_{runid}-checkpoint_{timestamp}.pt")
        #torch.save(checkpoint, f"./test-checkpoint_{timestamp}.pt")
        print(f"Saved checkpoint ./checkpoints/benchmark_1049inputs_{runid}-checkpoint_{timestamp}.pt", f"({elapsed_min:.1f} min)")
        #os.system("wandb sync --include-offline --sync-all wandb")
# 5) save a single timestamped checkpoint
elapsed_min = (time.time() - t0) / 60
timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint = {
    "epoch":         epoch+1,
    "elapsed_min":   elapsed_min,
    "net":           egnn_net.state_dict(),
    "mha":           mha_layer.state_dict(),
    "model":         protein_egnn.state_dict(),
    "convs":         [conv.state_dict(), nconv.state_dict()],
    "lin":           pred_head.state_dict(),
    "rbf":           rbf_layer.state_dict(),
    "optimizer":     optimizer.state_dict(),
    "scheduler":     scheduler.state_dict(),
    "train_history": tloss,
    "val_history":   vloss,
    "config":        config,
}
torch.save(checkpoint, f"../../../data/all_checkpoints/think_{runid}-checkpoint_{timestamp}.pt")
#torch.save(checkpoint, f"./test-checkpoint_{timestamp}.pt")
print(f"Saved checkpoint ../../../data/all_checkpoints/think_{runid}-checkpoint_{timestamp}.pt", elapsed_min)
#os.system("wandb sync --include-offline --sync-all wandb")


#from matplotlib import pyplot as plt
#plt.plot(vloss)
#plt.plot(tloss)


inputs dir: ../../../data/pkegnn_INS/inputs/*.npz
Whole model          : 8,300
{'runid': '20250722_070304-noH', 'num_nbrs': 100, 'num_nbrs_egnn': 11, 'learning_rate': [0.005], 'dataset': ('../../../data/pkegnn_INS/inputs/*.npz', 7, 30), 'epochs': 30, 'dim': 2, 'depth': 2, 'basis': 3, 'hidden_dim': 4, 'num_heads': 5, 'dropout': [0.01, 0.03], 'rbf cutoff': 20.0, 'loss': MSELoss(), 'params': 8300}
Epoch   0 | train L1 = 3.4387
              |  val L1 = 3.5404
Epoch   1 | train L1 = 3.3785
              |  val L1 = 3.5102
Epoch   2 | train L1 = 3.3216
              |  val L1 = 3.4797
Epoch   3 | train L1 = 3.3790
              |  val L1 = 3.4402
Epoch   4 | train L1 = 3.2455
              |  val L1 = 3.3964
Epoch   5 | train L1 = 3.1465
              |  val L1 = 3.3739
Epoch   6 | train L1 = 3.1250
              |  val L1 = 3.2656
Epoch   7 | train L1 = 3.0110
              |  val L1 = 3.1852
Epoch   8 | train L1 = 2.9403
              |  val L1 = 3.0988
Epoch   9 | train L1 = 2.8295
     

In [6]:
import datetime, time
#from architecture import *
#model=load_model(N_NEIGHBORS)
#loaders=hoods(INPUT_DIR,N_NEIGHBORS)

import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os

from egnn_pytorch import EGNN_Network
import torch
import torch.nn as nn

import math

#EGNN

class EGNNBlock(nn.Module):
    """todo: try to take head out here
    egnn_net --> layer norm --> ffn head"""
    def __init__(self, dim, depth,hidden_dim,dropout,
                 num_positions, num_tokens,
                 num_nearest_neighbors,
                 norm_coors):
        super().__init__()
        self.egnn = EGNN_Network(
            dim=dim, depth=depth, dropout=dropout,
            num_positions=num_positions,
            num_tokens=num_tokens,
            num_nearest_neighbors=num_nearest_neighbors,
            norm_coors=norm_coors
        )
        self.norm1 = nn.LayerNorm(dim)
        self.ffn = nn.Sequential(
            nn.Linear(dim, hidden_dim*dim),
            nn.PReLU(),# LU(),
            nn.Linear(hidden_dim*dim, dim),
        )
        self.norm2 = nn.LayerNorm(dim)
    def forward(self, z, x):
        (h_list, coords) = self.egnn(z, x)
        h = h_list # [B,N,dim]
        h2 = h
        h  = self.norm1(h + h2)
        h2 = self.ffn(h)
        h  = self.norm2(h + h2)
        return [h], coords

class StackedEGNN(nn.Module):
    """TODO understand depth"""
    def __init__(self, dim, depth, hidden_dim, dropout,
                 num_positions, num_tokens,
                 num_nearest_neighbors,
                 norm_coors):
        super().__init__()
        self.blocks = nn.ModuleList([
            EGNNBlock(dim, depth, hidden_dim, dropout,
                      num_positions, num_tokens,
                      num_nearest_neighbors,
                      norm_coors)
            for _ in range(1)
        ])
    def forward(self, z, x):
        coords = x
        h_list = None
        for block in self.blocks:
            if h_list is None:
                h_list, coords = block(z, x)
            else:
                h_list, coords = block(z, coords)
        return h_list, coords

# --- RBF with learnable cutoff ---
class LearnableRBF(nn.Module):
    """TODO change cutout"""
    def __init__(self, num_basis=16, cutoff=10.0):
        super().__init__()
        #self.pairwise = torch.norm(x.unsqueeze(1) - x.unsqueeze(0), dim=-1)
        self.cutoff = nn.Parameter(torch.tensor(cutoff))
        self.mu     = nn.Parameter(torch.linspace(0.0, 1.0, num_basis))
        self.gamma  = nn.Parameter(torch.tensor(12.0))
    
    def pairwise_distances(self, dist):
        return torch.norm(dist.unsqueeze(1) - dist.unsqueeze(0), dim=-1)

    
    def forward(self, dist):
        # dist: [B,N,N]
        dist = self.pairwise_distances(dist)
        mu = self.mu * self.cutoff     # [K]
        d  = dist.unsqueeze(-1)        # [B,N,N,1]
        return torch.exp(-self.gamma * (d - mu)**2)


#Attn. note encoding dropout of 0.03
#TODO: SPECIFy max len and dropout.
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=1000, dropout=0.03):
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        pe = torch.zeros(max_len, embed_dim)
        pos = torch.arange(max_len, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, embed_dim, 2, dtype=torch.float)
                        * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(pos * div)
        cosp = torch.cos(pos * div)
        pe[:, 1::2] = cosp[:, : pe[:, 1::2].shape[1]]
        self.register_buffer('pe', pe.unsqueeze(1))
    def forward(self, x):
        return self.dropout(x + self.pe[: x.size(0)])

# --- Transformer‐style AttentionBlock ---
class AttentionBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, hidden_dim):
        super().__init__()
        self.encoding = PositionalEncoding(embed_dim)
        self.attn  = nn.MultiheadAttention(embed_dim, num_heads)
        self.norm1 = nn.LayerNorm(embed_dim)
        self.ffn   = nn.Sequential(
            nn.Linear(embed_dim, embed_dim * hidden_dim),
            nn.PReLU(),
            nn.Linear(embed_dim * hidden_dim, embed_dim),
        )
        self.norm2 = nn.LayerNorm(embed_dim)
        
    def forward(self, x, key_padding_mask=None):
        # x: [seq_len, batch, embed_dim]
        x = self.encoding(x)
        a, _ = self.attn(x, x, x, key_padding_mask=key_padding_mask)
        x    = self.norm1(x + a)
        f    = self.ffn(x)
        x    = self.norm2(x + f)
        return x,_






import datetime, time
#from architecture import *
import torch
import glob, math, time, datetime
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from egnn_pytorch import EGNN_Network
from torch.optim.lr_scheduler import ReduceLROnPlateau
import os
from egnn_pytorch import EGNN
#from pkegnn import *
# 0) start timer
t0 = time.time()
N_NEIGHBORS = 100
BATCH_SIZE  =  1         # not safe to increase
PIN_MEMORY  = torch.cuda.is_available()
# reproducibility + device
torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
conv=nn.Conv1d(1, 1, 7, padding=3).to(device)
# decide AMP only on GP0
use_amp = (device.type == "cuda")
if use_amp:
    scaler = GradScaler()
else:
    class DummyCM:
        def __enter__(self): pass
        def __exit__(self, *args): pass
    autocast = DummyCM
    scaler   = None

# 3) instantiate everything
dim, basis = 2, 3 #scale to 3,16 at least # dim must be divisible by 2
depth=2 #scale to 2, at least
hidden_dim=2
dropout=0.01
cutoff=20.0
num_neighbors=11



runid=datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "-noH"


from sklearn.neighbors import NearestNeighbors
from torch.utils.data import Dataset, DataLoader
import numpy as np, torch, glob

class InMemoryHoodDataset(Dataset):
    """
    Loads *.npz files, constructs fixed-size neighbourhoods around each
    site (anchor) and stores the result entirely in RAM.

    For a protein with S sites the shapes are
        z   : (S, N_NEIGHBORS)      int32
        pos : (S, N_NEIGHBORS, 3)   float32
        y   : (S,)                  float32
    """
    def __init__(self, paths, n_neighbors=N_NEIGHBORS, pin_memory=PIN_MEMORY):
        super().__init__()
        self.data = []
        nbrs = NearestNeighbors(n_neighbors=n_neighbors, algorithm="brute")

        for p in paths:
            try:
                dat = np.load(p, allow_pickle=True)
                z_all   = dat["z"].astype(np.int32)        # (N,)
                pos_all = dat["pos"].astype(np.float32)    # (N,3)
                sites   = dat["sites"].astype(np.float32)  # (S,3)
                y       = dat["pks"].astype(np.float32)    # (S,)

                if len(sites) == 0:
                    continue  # skip empty entries

                nbrs.fit(pos_all)
                idx = nbrs.kneighbors(sites, return_distance=False)   # (S, N_NEIGHBORS)

                z_hood   = torch.from_numpy(z_all[idx])            # (S,N_NEIGHBORS)
                pos_hood = torch.from_numpy(pos_all[idx])          # (S,N_NEIGHBORS,3)
                y        = torch.from_numpy(y)                     # (S,)

                if pin_memory:
                    z_hood   = z_hood.pin_memory()
                    pos_hood = pos_hood.pin_memory()
                    y        = y.pin_memory()

                self.data.append((z_hood, pos_hood, y))
            except Exception as e:
                print(f"skipping {p}: {e}")

    def __len__(self):             return len(self.data)
    def __getitem__(self, idx):    return self.data[idx]

# ---------------------------------------------------------------------
# 2) collate function  -------------------------------------------------
# ---------------------------------------------------------------------
def pad_collate(batch):
    B      = len(batch)
    S_max  = max(item[0].shape[0] for item in batch)   # longest protein

    # --- allocate on CPU (no device=...) ---------------
    zs   = torch.zeros (B, S_max, N_NEIGHBORS ,   dtype=torch.int32 )
    pos  = torch.zeros (B, S_max, N_NEIGHBORS ,3, dtype=torch.float32)
    ys   = torch.full  ((B, S_max), float("nan"),  dtype=torch.float32)
    mask = torch.zeros (B, S_max,               dtype=torch.bool)

    for b,(z,pos_b,y) in enumerate(batch):
        S = z.shape[0]
        zs [b, :S] = z
        pos[b, :S] = pos_b
        ys [b, :S] = y
        mask[b, :S] = True

    return zs, pos, ys, mask


def count_params(model: torch.nn.Module, trainable_only: bool = True) -> int:
    """
    Returns the total number of (optionally trainable) parameters.
    """
    if trainable_only:
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    else:
        return sum(p.numel() for p in model.parameters())


# ---------------------------------------------------------------------
# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------

# 0) parameters you might want to expose at the top of the script
# ---------------------------------------------------------------------

# 3) data loaders ------------------------------------------------------
# ---------------------------------------------------------------------
np.random.seed(0)
INPUTS_DIR="../../../data/pkegnn_INS/inputs/*.npz"
print("inputs dir:", INPUTS_DIR)
all_paths = sorted(glob.glob(INPUTS_DIR))
#print(all_paths)
#split=0.75
#t=int(len(all_paths) *.75) - 1

#v=int(len(all_paths) *.25)
np.random.shuffle(all_paths)
train_paths, val_paths = all_paths[:2] + all_paths[5:10], all_paths[20:50]
#print(train_paths[0],val_paths[0])
nconv=nn.Conv1d(N_NEIGHBORS,1,dim+basis).to(device)
train_ds = InMemoryHoodDataset(train_paths)
val_ds   = InMemoryHoodDataset(val_paths)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE,
                          shuffle=True,  collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)
val_loader   = DataLoader(val_ds  , batch_size=BATCH_SIZE,
                          shuffle=False, collate_fn=pad_collate,
                          num_workers=0, pin_memory=PIN_MEMORY)


# ---------------------------------------------------------------------
# 4) model pieces ------------------------------------------------------
# ---------------------------------------------------------------------
egnn_net = StackedEGNN(dim=dim, depth=depth, hidden_dim=hidden_dim,
                       dropout=dropout, num_positions=1000, num_tokens=118,
                       num_nearest_neighbors=num_neighbors,
                       norm_coors=True).to(device)

rbf_layer = LearnableRBF(num_basis=basis, cutoff=cutoff).to(device)
mha_layer = AttentionBlock(embed_dim=dim + basis,
                           num_heads=dim + basis,
                           hidden_dim=hidden_dim).to(device)
pred_head = nn.Linear(1, 1).to(device)
#pred_head2 = nn.Linear(1, 1).to(device)

protein_egnn=EGNN(dim=1,update_coors=True, norm_coors=True, norm_feats=True, fourier_features=6, valid_radius=8)
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(
    list(egnn_net.parameters()) +
    list(rbf_layer.parameters()) +
    list(mha_layer.parameters()) +
    list(conv.parameters()) +
    list(nconv.parameters()) +

    list(pred_head.parameters()) +
    list(protein_egnn.parameters()),
    lr=5e-3
)
modell=[egnn_net, protein_egnn, nconv,rbf_layer,mha_layer,pred_head,conv] ####
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=.99, patience=0, cooldown=0, min_lr=1e-8, verbose=False)
epochs = 30 # or whatever you like
total_all    = count_params(torch.nn.ModuleList(
                    modell))


print(f"Whole model          : {total_all:,}")

config={"runid": runid,
        "num_nbrs": N_NEIGHBORS,
        "num_nbrs_egnn": num_neighbors,
        "learning_rate": [op["lr"] for op in optimizer.param_groups], #net mha model rbf
        "dataset": (INPUTS_DIR,len(train_ds),len(val_ds)),
        "epochs": epochs,
        "dim": dim,
        "depth": depth,
        "basis": basis,
        "hidden_dim":hidden_dim,
        "num_heads":dim+basis, #!
        "dropout": [dropout, 0.03], #egnn p.enc. 
        "rbf cutoff": cutoff,
        "loss": criterion,
        "params":total_all}
        #"architect":str(modell).encode()}

print(config)
# ---------------------------------------------------------------------
# 5) forward for a *compressed* batch (R residues, N neighbours)
# ---------------------------------------------------------------------
# ---------------------------------------------------------------------
# 5) single-path forward – no shape guessing, no branching
# ---------------------------------------------------------------------
def forward_residues(z_r, x_r):
    """
    z_r : (R, N)       int32   – atomic numbers for R residues
    x_r : (R, N, 3)    float32 – coordinates
    returns (R, dim + basis)   – per-residue embeddings
    """
    # ---------- EGNN ----------
    h_out, coords = egnn_net(z_r, x_r)          # h_out is [tensor] or tensor
    h = h_out[0] if isinstance(h_out, (list, tuple)) else h_out   # (R, N, dim)

    # ---------- RBF on *input* coords (already (R,N,3)) ----------
    #d   = torch.cdist(x_r, x_r)            # (R, N, N)
    rbf = rbf_layer(coords)                     # (R, N, N, basis)
    

    # ---------- concat & attention ----------
    h0  = h.transpose(1, 2)                # (R, dim,   N)
    r0  = rbf[:, 0].transpose(1, 2)        # (R, basis, N)
    tok = torch.cat((r0, h0), dim=1)       # (R, dim+basis, N)

    tok, _ = mha_layer(tok.permute(2, 0, 1))   # (N, R, C) → attn(+PE) #TODO suppress _
    tok=nconv(tok.to(device).permute(1,0,2)).to(device)
    #tok    = tok.permute(1, 0, 2).max(dim=1).values   # (R, C) max over neighbours
    return tok,     coords.mean(dim=1).unsqueeze(0)                               # (R, dim + basis)
                                         # (R, dim+basis)

# ---------------------------------------------------------------------
# 6) training / validation loop ---------------------------------------
# ---------------------------------------------------------------------
tloss, vloss = [], []
for epoch in range(epochs):
    # ======== TRAIN ========
    egnn_net.train(); rbf_layer.train(); mha_layer.train(); protein_egnn.train(); conv.train(); nconv.train();pred_head.train(); #p#red_head2.train(); 
    tr_losses = []

    for z, x, y, mask in train_loader:                 # z:(B,S,N)  mask:(B,S)
        # compress away padding →  (R, N), (R, N, 3), (R,)
        valid      = mask.view(-1)
        z_res      = z.view(-1, z.size(2))[valid].to(device)
        x_res      = x.view(-1, x.size(2), 3)[valid].to(device)
        y_res      = y.view(-1)[valid].to(device)

        optimizer.zero_grad()

        #model
        feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        #print(feats)
        preds = pred_head(feats.to(device))    
        t         = preds.to(device)[:,0].T.unsqueeze(2)         # (1, R, 1) on GPU
        centroids = centroids.to(device)  # (1, R, 3) on GPU

        protein_egnn = protein_egnn.to(device)             # make sure weights are there
        preds = protein_egnn(t, centroids)[0].permute(1,2,0)
        #preds = pred_head2(preds.to(device))
        #loss  = criterion(preds.flatten(), y_res)
        with autocast():
                preds=conv(preds).to(device)

                loss  = criterion(preds.flatten(), y_res)
            

        if use_amp:
            scaler.scale(loss).backward()
            scaler.step(optimizer); scaler.update()
        else:
            loss.backward(); optimizer.step()

        tr_losses.append(loss.item())
    tmean=np.mean(tr_losses)

    print(f"Epoch {epoch:3d} | train L1 = {tmean:.4f}")
    tloss.append(tmean.item())
    #elapsed_min = (time.time() - t0) / 60
    
    # ======== VALID ========
    egnn_net.eval(); rbf_layer.eval(); mha_layer.eval(); protein_egnn.eval(); conv.eval(); nconv.eval();pred_head.eval(); #pred_head2.eval(); 
    vl_losses = []

    with torch.no_grad():
        for z, x, y, mask in val_loader:
            valid   = mask.view(-1)
            z_res   = z.view(-1, z.size(2))[valid].to(device)
            x_res   = x.view(-1, x.size(2), 3)[valid].to(device)
            y_res   = y.view(-1)[valid].to(device)

                #model
            feats, centroids = forward_residues(z_res, x_res)         # (R, C)
        
            preds = pred_head(feats.to(device))    
            t         = preds.to(device)[:,0].T.unsqueeze(2)         # (1, R, 1) on GPU
            centroids = centroids.to(device)  # (1, R, 3) on GPU

            protein_egnn = protein_egnn.to(device)             # make sure weights are there
            preds = protein_egnn(t, centroids)[0].permute(1,2,0)
            #preds = pred_head2(preds.to(device))
            with autocast():
                preds=conv(preds).to(device)

                loss  = criterion(preds.flatten(), y_res)
            vl_losses.append(loss) ###his is so inefficient. batch the proteins and for free bye cpu

    #elapsed_min = (time.time() - t0) / 60
    if vl_losses: #cpu
        vl=torch.mean(torch.stack(vl_losses))
        print(f"              |  val L1 = {vl.item():.4f}")
    else:
        print("E70")
        
    #L=torch.mean(torch.stack(vl_losses))
    scheduler.step(vl)
    vloss.append(vl.item())
    
    if epoch in [100,200,300,400,500,600,700,800,900]: #TODO doesnt work
        # 5) save a single timestamped checkpoint
        elapsed_min = (time.time() - t0) / 60
        timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        checkpoint = {
            "epoch":         epoch+1,
            "elapsed_min":   elapsed_min,
            "net":           egnn_net.state_dict(),
            "mha":           mha_layer.state_dict(),
            "model":         protein_egnn.state_dict(),
            "convs":         [conv.state_dict(), nconv.state_dict()],
            "lin":           pred_head.state_dict(),
            "rbf":           rbf_layer.state_dict(),
            "optimizer":     optimizer.state_dict(),
            "scheduler":     scheduler.state_dict(),
            "train_history": tloss,
            "val_history":   vloss,
            "config":        config,
        }
        torch.save(checkpoint, f"./checkpoints/benchmark_1049inputs_{runid}-checkpoint_{timestamp}.pt")
        #torch.save(checkpoint, f"./test-checkpoint_{timestamp}.pt")
        print(f"Saved checkpoint ./checkpoints/benchmark_1049inputs_{runid}-checkpoint_{timestamp}.pt", f"({elapsed_min:.1f} min)")
        #os.system("wandb sync --include-offline --sync-all wandb")
# 5) save a single timestamped checkpoint
elapsed_min = (time.time() - t0) / 60
timestamp   = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
checkpoint = {
    "epoch":         epoch+1,
    "elapsed_min":   elapsed_min,
    "net":           egnn_net.state_dict(),
    "mha":           mha_layer.state_dict(),
    "model":         protein_egnn.state_dict(),
    "convs":         [conv.state_dict(), nconv.state_dict()],
    "lin":           pred_head.state_dict(),
    "rbf":           rbf_layer.state_dict(),
    "optimizer":     optimizer.state_dict(),
    "scheduler":     scheduler.state_dict(),
    "train_history": tloss,
    "val_history":   vloss,
    "config":        config,
}
torch.save(checkpoint, f"../../../data/all_checkpoints/think_{runid}-checkpoint_{timestamp}.pt")
#torch.save(checkpoint, f"./test-checkpoint_{timestamp}.pt")
print(f"Saved checkpoint ../../../data/all_checkpoints/think_{runid}-checkpoint_{timestamp}.pt", elapsed_min)
#os.system("wandb sync --include-offline --sync-all wandb")


#from matplotlib import pyplot as plt
#plt.plot(vloss)
#plt.plot(tloss)


inputs dir: ../../../data/pkegnn_INS/inputs/*.npz
Whole model          : 8,170
{'runid': '20250722_064838-noH', 'num_nbrs': 100, 'num_nbrs_egnn': 11, 'learning_rate': [0.005], 'dataset': ('../../../data/pkegnn_INS/inputs/*.npz', 7, 30), 'epochs': 30, 'dim': 2, 'depth': 2, 'basis': 3, 'hidden_dim': 2, 'num_heads': 5, 'dropout': [0.01, 0.03], 'rbf cutoff': 20.0, 'loss': MSELoss(), 'params': 8170}
Epoch   0 | train L1 = 3.4201
              |  val L1 = 3.5333
Epoch   1 | train L1 = 3.3301
              |  val L1 = 3.5031
Epoch   2 | train L1 = 3.3419
              |  val L1 = 3.4873
Epoch   3 | train L1 = 3.3260
              |  val L1 = 3.4646
Epoch   4 | train L1 = 3.2425
              |  val L1 = 3.4168
Epoch   5 | train L1 = 3.1919
              |  val L1 = 3.4210
Epoch   6 | train L1 = 3.1190
              |  val L1 = 3.3061
Epoch   7 | train L1 = 3.0348
              |  val L1 = 3.2341
Epoch   8 | train L1 = 2.9286
              |  val L1 = 3.1698
Epoch   9 | train L1 = 2.8849
     